In [16]:
#basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
#pytorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
#functions file
from functions import *
#counting time
import time

import scipy.ndimage.filters as filters
from skimage.feature import peak_local_max

In [2]:
# reload the model
# if you used a customized model you need to change the class below

# class
class Pixel14(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(2, 21, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(21, 42, kernel_size=3)
        n = 168
        self.fc1 = torch.nn.Linear(168,n,bias=True)
        self.fc2 = torch.nn.Linear(n,n,bias=True)
        self.fc3 = torch.nn.Linear(n,n,bias=True)
        self.fc4 = torch.nn.Linear(n,n,bias=True)
        self.fc5 = torch.nn.Linear(n,n,bias=True)
        # out layer
        self.fcOut = torch.nn.Linear(n, 1,bias=True)
      
    def forward(self, x):
        relu = torch.nn.functional.relu
        max_pool2d = torch.nn.functional.max_pool2d
        # convolutions
        x = relu(max_pool2d(self.conv1(x),2))
        x = relu(max_pool2d(self.conv2(x),2))
        x = x.view(-1,168)
        # linear layers
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = relu(self.fc3(x))
        x = relu(self.fc4(x))
        x = relu(self.fc5(x))
        
        
        # out layer
        x = self.fcOut(x)
        return torch.sigmoid(x)

path_to_model = 'models/my_model'

load = torch.load(path_to_model)

In [11]:
#parameter 
thr_permissivity = 0.005    #-> proportion of pixel kept in the image by quantile selction (foreground selection)

thr_predictions = 0.05      #-> threshold to filter the prediction map
                            #-> better to check the best value with findBestThr

In [5]:
### Load images

# import images + the corresponding names
## Load using channels
channels = [0,1,2]
images,names = loadAllCtrImages(channels=channels, format_="tif", path = path)

print( 'number of images ' + str(len(images)) )

number of images 247


In [6]:
## load using a single folder (a single type of fluorescence)
# example with CP110
path = 'data/RPE1wt_CP110+GTU88+PCNT_2/CP110/tif'

images = []
names = [f for f in listdir(path) if isfile(join(path, f))]

for n in names:
    images.append( cv.imread( path + '/' + n , cv.IMREAD_UNCHANGED) )

names = [n[:-4] for n in names]

print( 'number of images ' + str(len(images)) )

number of images 25


In [ ]:
## uncomment if you want to use a certain number of random images in the list
'''
spl_nb = 20

indexes_spl = [ i for i in range(len(images)) ]
np.random.shuffle( indexes_spl )

spl_images = []
spl_names = []

for i in indexes_spl[:spl_nb]:
    spl_images.append( images[i] )
    spl_names.append( names[i] )
    
images = spl_images
names = spl_names
'''

In [8]:
# generate the predictions map using the model
list_predictions = generatePredictions( normalizeImages( images ), load, thr_permissivity )

In [22]:
df = generateDataFrame( list_predictions, names, thr_predictions )

In [ ]:
#save the dataframe
save_name = 'my_predictions.csv'
df.to_csv( save_name )